In [1]:
import os
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import requests
import time

In [2]:
!which chromedriver #install chrome driver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [5]:
browser.visit(url)
time.sleep(1)

In [6]:
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
titles=soup.find_all('div', class_="content_title")
#print(titles)

In [8]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [9]:
latest_title = titles[1]
latest_title = latest_title.a.text
print(latest_title)

NASA Engineers Checking InSight's Weather Sensors


In [10]:
#scrape latest news paragraph text
paragraph=soup.find('div', class_='article_teaser_body')
latest_paragraph=paragraph.text
print(latest_paragraph)

An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


In [11]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [12]:
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
results=soup.prettify()
#print(results)

In [14]:
#featured_image_url
images= soup.find_all('div', class_="img")
#print(images)

In [15]:
source_url = images[0].img["src"]
#print(source_url)

In [16]:
#url for first image
featured_image_url =(f"https://www.jpl.nasa.gov{source_url}")
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24077-640x350.jpg


In [17]:
url="https://space-facts.com/mars/"
tables = pd.read_html(url)
#tables

In [18]:
tables = pd.read_html(str(url))[0]
tables

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [19]:
url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [20]:
html = browser.html
soup = bs(html, 'html.parser')

In [21]:
#print(soup.prettify())

In [22]:
images=soup.find_all('a', class_="itemLink product-item")
#print(images)

In [23]:
image_title=soup.find_all('h3')
image_title

[<h3>Cerberus Hemisphere Enhanced</h3>,
 <h3>Schiaparelli Hemisphere Enhanced</h3>,
 <h3>Syrtis Major Hemisphere Enhanced</h3>,
 <h3>Valles Marineris Hemisphere Enhanced</h3>]

In [24]:
#print title as text
image_title[1].text

'Schiaparelli Hemisphere Enhanced'

In [25]:
#first url
url=images[6]['href']
click_url=f"https://astrogeology.usgs.gov/{url}"
click_url

'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced'

In [26]:
browser.visit(click_url)
html=browser.html
soup=bs(html,'html.parser')

In [27]:
#grab url of image on next page
#after clicking hemisphere title
source_url=soup.find('li')
picture_url=source_url.a['href']
picture_url

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'

In [28]:
#create dictionary with hemisphere title and url to enhanced picture
title_list=[]
url_list=[]
dictionary=[]
images = images[: : 2]
for i in range(len(image_title)):
    title_list.append(image_title[i].text)
    url=images[i]['href']
    click_url=f"https://astrogeology.usgs.gov/{url}"
    browser.visit(click_url)
    time.sleep(1)
    html=browser.html
    soup=bs(html,'html.parser')
    source_url=soup.find('li')
    picture_url=source_url.a['href']
    url_list.append(picture_url)
    dictionary.append({"title":image_title[i].text, "img_url":picture_url})

In [29]:
#print(title_list)

In [30]:
#print(url_list)

In [31]:
print(dictionary)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
